In [1]:
#Se importan todas las librerías, tanto de análisis de sentimientos, como traducción y pandas
import nltk
import pandas as pd
from translate import Translator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize
from nltk.corpus import stopwords
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
#Se descargan los packs de léxico y puntuación
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#Se realiza el web scrapping mediante un bucle for para poder sacar todas opiniones de todas las páginas de booking
tables_info=[]
for i in range (1,10):
    url= "https://www.booking.com/reviews/es/hotel/flamero.es.html?label=gen173nr-1FCA0oRkIHZmxhbWVyb0gKWARoRogBAZgBCrgBF8gBDNgBAegBAfgBAogCAagCA7gCrfCeqQbAAgHSAiRhODg5OWY3NC00MzE4LTQ1ZjQtOGI5Mi0yZGFlYmRmZWM0NjTYAgXgAgE&sid=d9ddf77f00ff19eea1916cb4305df56a&customer_type=total&hp_nav=0&old_page=0&order=featuredreviews&page="+str(i)+"&r_lang=es&rows=75&"
    headers = {"User-Agent":"Mozilla/5.0"}
    req = Request(url,headers=headers)
    raw_web = urlopen(req).read()
    soup = BeautifulSoup(raw_web, 'html.parser')
    tables_info.append(soup.find_all('span', attrs={"itemprop": "reviewBody"}))

In [4]:
#Se guardan las reseñas en una lista y se limpian mediante el comando text.strip
reseñas=[]
for fil in range(0,9):
    for j in tables_info[fil]:
        reseñas.append(j.text.strip())

#Limitamos a 210 las reseñas, que son las que nos permite traducir la librería translate
reseñas=reseñas[:210]
reseñas

['Hacer cola para cenar no le gusta a nadie, pero es cierto que el hotel estaba lleno, no pasa nada por esperar tu turno 15 minutitos.',
 'Esperaba menos calidad y sorprende a cada momento. Detallazo el que tuvieron con mi mujer y conmigo por nuestro aniversario consiguiéndonos una habitación con vistas al mar.',
 'Piscina muy pequeña',
 'Cercanía playa',
 'El buffet no era gran cosa',
 'Las instalaciones',
 'En general la experiencia fue genial. El hotel lo tienen muy bonito, hall, bar, comedor, bastante nuevo y actual. La habitación con vistas al mar espectacular, nos ofrecieron una habitacion superior (deluxe vista mar creo que era) era solo un poco mas cara ... pero después de estar la verdad es que el cambio mereció la pena.  La limpieza perfecta y el personal encantador y muy pendiente de todo. En el bufet muy bien, había mucha variedad y todo lo que comimos estaba muy bueno.',
 'Habitación espectacular. Cama muy cómoda',
 'Ubicación, habitación con vistas espectaculares, limpiez

In [7]:
#Para poder usar la librería nltk hace falta traducir las frases por lo que se le pasan todas las reseñas
#y se guardan en otra lista llamada frases ingles
frases_ingles = []
for i in reseñas:
    translator=Translator(from_lang='es',to_lang='en')
    traduccion=translator.translate(i)
    frases_ingles.append(traduccion)
frases_ingles

["No one likes queuing for dinner, but it's true that the hotel was full, it's okay to wait for your turn 15 minutes.",
 'I expected less quality and it surprises every moment. I detail what they had with my wife and me for our anniversary, getting us a room with sea views.',
 'Very small pool',
 'Near the beach',
 'The buffet was no big deal',
 'The facilities',
 'Overall the experience was great. The hotel is very nice, hall, bar, dining room, quite new and current. The room with spectacular sea views, we were offered a superior room (deluxe sea view I think it was) was only a little more expensive ... but after being the truth is that the change was worth it.  The perfect cleanliness and the charming and very attentive staff. The buffet was very good, there was a lot of variety and everything we ate was very good.',
 'Spectacular room. Very comfortable bed',
 'Location, room with spectacular views, cleanliness and staff attention',
 'The buffet waiters are very edgy. The crowding of

In [8]:
#Convertimos la lista en una tabla
frases_ingles= pd.DataFrame({'Reseñas':frases_ingles})

In [9]:
#Guardamos la tabla en un csv
frases_ingles.to_csv('Outputs/frases_ingles.csv',index=False)

In [5]:
#Recuperamos la tabla
frases_ingles = pd.read_csv('Outputs/frases_ingles.csv')
frases_ingles

,Reseñas
0,"No one likes queuing for dinner, but it's true..."
1,I expected less quality and it surprises every...
2,Very small pool
3,Near the beach
4,The buffet was no big deal
...,...
205,La ubicación muy buena.
206,"Location, cleanliness and attention of staff"
207,"The bathroom smelled like plumbing, a lot of n..."
208,Location


In [6]:
#Estas son las listas con palabras positivas y negativas que tiene en cuenta el script para poder apoyar a los comentarios que detecte como neutros
palabras_positivas = ["good","happy","big","recommend","nice","great", "excellent", "enjoy", "enjoyed", "perfect", "fabulous", "elegant", "delicious", "unforgettable", "spectacular", "highly","okay", "beautiful"]
palabras_negativas = ["old","poor","small","uncomfortable","bad","slow", "shit", "suffered", "horrible", "disaster","no","wasn't","didn't","noisy","expensive","missing", "lack"]

#Además tambien le pasamos listas con palabras para poder categorizar y sacar las categorias deseadas
categoria_limpieza = ["clean","tidy", "dirt", "cleaner", "towels", "rat", "rats", "cockroach", "cockroachs", "cleaner", "cleaners","cleaning", "cleanliness"]
categoria_instalaciones = ["pool","stairs","reception","elevator", "elevators", "lobby", "gym", "door", "doors", "water", "parking","park","facilities","microwave","bar","hall","fridge","park"]
categoria_habitacion = ["shower","studio","beds","pillows","conditioning","bed","room", "bathroom","rooms", "kitchen","fridge", "suite", "suites","bathroom", "toilet", "bedroom", "bedrooms", "towels", "roomy", "spacious", "bright", "luminous", "mattress", "terrace"]
categoria_ubicacion = ["ubication","location","place","view","views","visits", "beach", "sea", "preserve", "reserve","light","noise"]
categoria_atencion = ["needs", "requirements", "staff", "reception", "support", "help", "care", "careless", "careful", "attendance", "gentle", "charm", "helpful", "attentive", "receptionist", "waiter", "waiters", "cleaner", "cleaners", "service", "employees", "friendly", "unfriendly", "kind", "kindly", "friendliness"]
categoria_ambiente = ["quiet", "noise", "noisy", "relax", "chill", "privacy"]
categoria_comida = ["food", "foods", "buffet", "breakfast", "dinner",  "lunch", "lunches", "fruit", "fruits", "gastronomy", "hungry", "meat", "fish", "bread", "delicious","meals", "meal"]

In [7]:
#Inicializamos el analizador de sentimiento
sia = SentimentIntensityAnalyzer()

In [8]:
#Sacamos los scores de las reseñas
scores=[]
for i in frases_ingles['Reseñas']:
    scores.append(sia.polarity_scores(i)['compound'])
scores

[0.7469,
 0.2263,
 0.0,
 0.0,
 -0.296,
 0.0,
 0.9848,
 0.5563,
 0.0,
 -0.4215,
 0.0,
 -0.5699,
 0.0,
 -0.7783,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2263,
 0.2732,
 0.0,
 -0.3818,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.4215,
 0.796,
 0.0,
 0.0,
 0.0,
 0.4588,
 -0.5765,
 0.4215,
 0.0,
 0.0,
 0.2862,
 0.6115,
 0.0,
 0.0,
 0.0,
 0.796,
 0.0,
 0.4927,
 0.0,
 0.0,
 0.0,
 0.4588,
 0.0,
 0.0,
 -0.8563,
 0.7269,
 0.0,
 0.0,
 -0.2411,
 0.0,
 0.0,
 0.3384,
 -0.7642,
 0.0,
 -0.296,
 0.0,
 0.25,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.296,
 0.0,
 0.0,
 0.875,
 0.0,
 0.4215,
 0.5719,
 0.0,
 0.0,
 0.807,
 -0.3182,
 0.6249,
 0.5719,
 0.0,
 0.4404,
 0.0,
 0.0,
 -0.4201,
 0.762,
 0.0,
 0.4939,
 -0.3182,
 0.0,
 0.4215,
 0.0,
 -0.2023,
 0.5574,
 0.2023,
 0.3384,
 0.0,
 0.0,
 0.4588,
 0.4754,
 0.0,
 0.0,
 0.0,
 0.1999,
 0.8999,
 -0.5849,
 0.0,
 0.0,
 0.0,
 -0.4449,
 -0.103,
 0.9483,
 0.0,
 0.0,
 -0.6968,
 0.7992,
 -0.6075,
 0.822,
 -0.4019,
 0.6486,
 0.9182,
 0.4215,
 0.0,
 0.5994,
 -0.2411,
 0.4927,
 0.0,
 0.0,
 0.0,
 0.7178,
 0.49

In [9]:
#Función mediante la cual la librería nltk tokeniza las frases y si alguna de esas palabras se encuentran dentro de la lista
#de palabras negativas o positivas se suma 1 a la categoria correspondiente
def calcular_puntuacion_sentimiento(frase_ingles):
    tokens = nltk.word_tokenize(frase_ingles)
    puntuacion_sentimiento = 0

    for token in tokens:
        if token in palabras_positivas:
            puntuacion_sentimiento += 1
        elif token in palabras_negativas:
            puntuacion_sentimiento -= 1

    return puntuacion_sentimiento

In [10]:
#Función para poder categorizar las frases, se devuelve una lista con el valor o los valores de la categoría,
#si se devuelve en blanco la frase se define como categoria general
def calcular_categoria_sentimiento(frase_ingles):
            tokens = nltk.word_tokenize(frase_ingles)
            categorias=[]
            cuenta_ubicacion = 0
            cuenta_habitacion = 0
            cuenta_limpieza = 0
            cuenta_instalaciones = 0
            cuenta_atencion = 0
            cuenta_ambiente = 0
            cuenta_comida = 0

            for token in tokens:
                token=token.lower()
                if token in categoria_ubicacion and cuenta_ubicacion == 0:
                    categorias.append('Ubicación')
                    cuenta_ubicacion = 1
                elif token in categoria_habitacion and cuenta_habitacion == 0:
                    categorias.append('Habitación')
                    cuenta_habitacion = 1
                elif token in categoria_limpieza and cuenta_limpieza == 0:
                    categorias.append('Limpieza')
                    cuenta_limpieza = 1
                elif token in categoria_instalaciones and cuenta_instalaciones == 0:
                    categorias.append('Instalaciones')
                    cuenta_instalaciones = 1
                elif token in categoria_atencion and cuenta_atencion == 0:
                    categorias.append('Atención al cliente')
                    cuenta_atencion = 1
                elif token in categoria_ambiente and cuenta_ambiente == 0:
                    categorias.append('Ambiente')
                    cuenta_ambiente = 1
                elif token in categoria_comida and cuenta_comida == 0:
                    categorias.append('Comida')
                    cuenta_comida = 1

            return categorias

In [11]:
#Creamos un listado con el sentimiento para cada reseña
sentimientos=[]
for i in frases_ingles['Reseñas']:
    i=i.lower()
    sentimiento = sia.polarity_scores(i)
    if sentimiento['compound'] >= 0.05:
        sentimientos.append(1)
    elif sentimiento['compound'] <= -0.05:
        sentimientos.append(-1)
    else: #Si el resultado es neutro, se calcula con la lista de palabras positivas o negativas, y si tampoco se encuentra en esta lista, se categoria ya como neutra
        puntuacion = calcular_puntuacion_sentimiento(i)
        if puntuacion > 0:
            sentimientos.append(1)
        elif puntuacion < 0:
            sentimientos.append(-1)
        else:
            sentimientos.append(0)
len(sentimientos)

210

In [12]:
#Creamos los listados para cada categoría
ubicacion=[]
habitacion=[]
limpieza=[]
instalaciones=[]
atencion=[]
ambiente=[]
comida=[]

#Añadimos las categorías por reseña para tener un One Hot Encoding
for i in frases_ingles['Reseñas']:
    categorias=calcular_categoria_sentimiento(i)
    
    if 'Ubicación' in categorias:
        ubicacion.append(1)
    else:
        ubicacion.append(0)
    
    if 'Habitación' in categorias:
        habitacion.append(1)
    else:
        habitacion.append(0)
        
    if 'Limpieza' in categorias:
        limpieza.append(1)
    else:
        limpieza.append(0)
    
    if 'Instalaciones' in categorias:
        instalaciones.append(1)
    else:
        instalaciones.append(0)
        
    if 'Atención al cliente' in categorias:
        atencion.append(1)
    else:
        atencion.append(0)
    
    if 'Ambiente' in categorias:
        ambiente.append(1)
    else:
        ambiente.append(0)
        
    if 'Comida' in categorias:
        comida.append(1)
    else:
        comida.append(0)

In [13]:
#Creamos la tabla
tabla=pd.DataFrame({'reseña':reseñas,'score':scores,'sentimientos': sentimientos,'Ubicación': ubicacion, 
                    'Habitación':habitacion, 'Limpieza': limpieza, 'Instalaciones': instalaciones, 
                    'Atención al cliente':atencion, 'Ambiente':ambiente, 'Comida': comida})
tabla

,reseña,score,sentimientos,Ubicación,Habitación,Limpieza,Instalaciones,Atención al cliente,Ambiente,Comida
0,"Hacer cola para cenar no le gusta a nadie, per...",0.7469,1,0,0,0,0,0,0,1
1,Esperaba menos calidad y sorprende a cada mome...,0.2263,1,1,1,0,0,0,0,0
2,Piscina muy pequeña,0.0000,-1,0,0,0,1,0,0,0
3,Cercanía playa,0.0000,0,1,0,0,0,0,0,0
4,El buffet no era gran cosa,-0.2960,-1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
205,La ubicación muy buena,0.0000,0,0,0,0,0,0,0,0
206,"Ubicación, la limpieza y atención del personal",0.0000,0,1,0,1,0,1,0,0
207,"El baño olía a tuberia, mucho ruido.",0.3612,1,1,1,0,0,0,0,0
208,Ubicación,0.0000,0,1,0,0,0,0,0,0


In [14]:
#Guardamos la tabla
tabla.to_csv('Outputs/reseñas_sentimientos_categorias.csv',index=False)

In [15]:
#Creamos los contadores combinando categorías con sentimiento
opiniones_positivas_ubicacion=0
opiniones_negativas_ubicacion=0
opiniones_neutras_ubicacion=0

opiniones_positivas_limpieza=0
opiniones_negativas_limpieza=0
opiniones_neutras_limpieza=0

opiniones_positivas_instalaciones=0
opiniones_negativas_instalaciones=0
opiniones_neutras_instalaciones=0

opiniones_positivas_habitacion=0
opiniones_negativas_habitacion=0
opiniones_neutras_habitacion=0

opiniones_positivas_atencion=0
opiniones_negativas_atencion=0
opiniones_neutras_atencion=0

opiniones_positivas_comida=0
opiniones_negativas_comida=0
opiniones_neutras_comida=0

opiniones_positivas_ambiente=0
opiniones_negativas_ambiente=0
opiniones_neutras_ambiente=0

opiniones_positivas=0
opiniones_negativas=0
opiniones_neutras=0

for j in frases_ingles['Reseñas']: #Se recorren todas las opiniones, se define hasta 210 ya que la librería de translate tiene límite de traducciones gratuitas cada 24 y el máximo permitido es 210
    categoria=0
    j=j.lower()
    categoria = calcular_categoria_sentimiento(j)
    sentimiento = sia.polarity_scores(j) #Primero se analiza con la librería NLTK si es positiva, negativa o neutra con el valor de la variable compound
    if sentimiento['compound'] >= 0.05:
        opiniones_positivas+=1
    elif sentimiento['compound'] <= -0.05:
        opiniones_negativas+=1
    else: #Si el resultado es neutro, se calcula con la lista de palabras positivas o negativas, y si tampoco se encuentra en esta lista, se categoria ya como neutra
        puntuacion = calcular_puntuacion_sentimiento(j)
        if puntuacion > 0:
            opiniones_positivas+=1
        elif puntuacion < 0:
            opiniones_negativas+=1
        else:
            opiniones_neutras+=1
            
    #Se recorren las listas clasificando cada frase dependiendo de su categoria
    for i in range(len(categoria)):
        if categoria[i]=='Ubicación':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_positivas_ubicacion+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_ubicacion+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_ubicacion+=1
                elif puntuacion < 0:
                    opiniones_negativas_ubicacion+=1
                else:
                    opiniones_neutras_ubicacion+=1
                        
        elif categoria[i]=='Habitación':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_positivas_habitacion+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_habitacion+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_habitacion+=1
                elif puntuacion < 0:
                    opiniones_negativas_habitacion+=1
                else:
                    opiniones_neutras_habitacion+=1
                        
        elif categoria[i]=='Limpieza':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_negativas_limpieza+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_limpieza+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_limpieza+=1
                elif puntuacion < 0:
                    opiniones_negativas_limpieza+=1
                else:
                    opiniones_neutras_limpieza+=1
                        
        elif categoria[i]=='Comida':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_negativas_comida+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_comida+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_comida+=1
                elif puntuacion < 0:
                    opiniones_negativas_comida+=1
                else:
                    opiniones_neutras_comida+=1
                        
        elif categoria[i]=='Ambiente':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_negativas_ambiente+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_ambiente+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_ambiente+=1
                elif puntuacion < 0:
                    opiniones_negativas_ambiente+=1
                else:
                    opiniones_neutras_ambiente+=1
                        
        elif categoria[i]=='Instalaciones':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_positivas_instalaciones+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_instalaciones+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_instalaciones+=1
                elif puntuacion < 0:
                    opiniones_negativas_instalaciones+=1
                else:
                    opiniones_neutras_instalaciones+=1
                        
        elif categoria[i]=='Atención al cliente':
            sentimiento = sia.polarity_scores(j)
            if sentimiento['compound'] >= 0.05:
                opiniones_negativas_atencion+=1
            elif sentimiento['compound'] <= -0.05:
                opiniones_negativas_atencion+=1
            else:
                puntuacion = calcular_puntuacion_sentimiento(j)
                if puntuacion > 0:
                    opiniones_positivas_atencion+=1
                elif puntuacion < 0:
                    opiniones_negativas_atencion+=1
                else:
                    opiniones_neutras_atencion+=1
                        
print("Categoria Ubicación: Hay",opiniones_positivas_ubicacion,"opiniones positivas. Hay",opiniones_negativas_ubicacion,"opiniones negativas. Hay",opiniones_neutras_ubicacion,"opiniones neutras")
print("Categoria Habitacion: Hay",opiniones_positivas_habitacion,"opiniones positivas. Hay",opiniones_negativas_habitacion,"opiniones negativas. Hay",opiniones_neutras_habitacion,"opiniones neutras")
print("Categoria Limpieza: Hay",opiniones_positivas_limpieza,"opiniones positivas. Hay",opiniones_negativas_limpieza,"opiniones negativas. Hay",opiniones_neutras_limpieza,"opiniones neutras")
print("Categoria Instalaciones: Hay",opiniones_positivas_instalaciones,"opiniones positivas. Hay",opiniones_negativas_instalaciones,"opiniones negativas. Hay",opiniones_neutras_instalaciones,"opiniones neutras")
print("Categoria Comida: Hay",opiniones_positivas_comida,"opiniones positivas. Hay",opiniones_negativas_comida,"opiniones negativas. Hay",opiniones_neutras_comida,"opiniones neutras")
print("Categoria Atención al cliente: Hay",opiniones_positivas_atencion,"opiniones positivas. Hay",opiniones_negativas_atencion,"opiniones negativas. Hay",opiniones_neutras_atencion,"opiniones neutras")
print("Categoria Ambiente: Hay",opiniones_positivas_ambiente,"opiniones positivas. Hay",opiniones_negativas_ambiente,"opiniones negativas. Hay",opiniones_neutras_ambiente,"opiniones neutras")
print("Total: Hay",opiniones_positivas,"opiniones positivas. Hay",opiniones_negativas,"opiniones negativas. Hay",opiniones_neutras,"opiniones neutras")

Categoria Ubicación: Hay 36 opiniones positivas. Hay 8 opiniones negativas. Hay 36 opiniones neutras
Categoria Habitacion: Hay 23 opiniones positivas. Hay 20 opiniones negativas. Hay 15 opiniones neutras
Categoria Limpieza: Hay 1 opiniones positivas. Hay 16 opiniones negativas. Hay 8 opiniones neutras
Categoria Instalaciones: Hay 18 opiniones positivas. Hay 21 opiniones negativas. Hay 21 opiniones neutras
Categoria Comida: Hay 0 opiniones positivas. Hay 42 opiniones negativas. Hay 11 opiniones neutras
Categoria Atención al cliente: Hay 1 opiniones positivas. Hay 23 opiniones negativas. Hay 3 opiniones neutras
Categoria Ambiente: Hay 0 opiniones positivas. Hay 3 opiniones negativas. Hay 0 opiniones neutras
Total: Hay 80 opiniones positivas. Hay 49 opiniones negativas. Hay 81 opiniones neutras


In [16]:
#Creamos la tabla
datos = {
    'Categoria': ['Ubicación', 'Habitacion', 'Limpieza', 'Instalaciones', 'Comida', 'Atención al cliente', 'Ambiente', 'Total'],
    'Opiniones Positivas': [opiniones_positivas_ubicacion, opiniones_positivas_habitacion, opiniones_positivas_limpieza, opiniones_positivas_instalaciones, opiniones_positivas_comida, opiniones_positivas_atencion, opiniones_positivas_ambiente, opiniones_positivas],
    'Opiniones Negativas': [opiniones_negativas_ubicacion, opiniones_negativas_habitacion, opiniones_negativas_limpieza, opiniones_negativas_instalaciones, opiniones_negativas_comida, opiniones_negativas_atencion, opiniones_negativas_ambiente, opiniones_negativas],
    'Opiniones Neutras': [opiniones_neutras_ubicacion, opiniones_neutras_habitacion, opiniones_neutras_limpieza, opiniones_neutras_instalaciones, opiniones_neutras_comida, opiniones_neutras_atencion, opiniones_neutras_ambiente, opiniones_neutras]
}
df = pd.DataFrame(datos)

In [17]:
df

,Categoria,Opiniones Positivas,Opiniones Negativas,Opiniones Neutras
0,Ubicación,36,8,36
1,Habitacion,23,20,15
2,Limpieza,1,16,8
3,Instalaciones,18,21,21
4,Comida,0,42,11
5,Atención al cliente,1,23,3
6,Ambiente,0,3,0
7,Total,80,49,81


In [18]:
#Guardamos la tabla
df.to_csv('Outputs/opiniones.csv',index=False)